In [ ]:
import numpy as np
import pandas as pd

from random import choice, randrange
from datetime import  date, timedelta, datetime
from sqlalchemy import func, asc

import networkx as nx
import networkx.algorithms.community as nx_comm

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from matplotlib import colors as mpl_colors
from mpl_toolkits.basemap import Basemap as Basemap
from tqdm import tqdm


from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.forecasting.stl import STLForecast
from statsmodels.tsa.seasonal import STL
from sklearn.metrics import  mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from scipy.stats import kstest, ttest_ind
from scipy.signal import periodogram

import torch.nn as nn
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

from lib_database import *


pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.options.display.expand_frame_repr = False

# Анализ судов

Анализ распределения судов по бассейнам судов 

In [ ]:
def various_bassin(df, min_calls=0, ship_type ='13 / Oil tanker', print_table=False, print_hist=False):
    df_pivot = pd.pivot_table(df[(df['Количество судозаходов']>2) & (df['Тип судна']=='13 / Oil tanker')],
                                index=['ship_id', 'Тип судна', 'Бассейн'],)
    
    if print_table:
        print(df_pivot)

    df_pivot = pd.pivot_table(df[(df['Количество судозаходов']>min_calls) & (df['Тип судна']==ship_type)],
                                index=['ship_id', 'Тип судна'],
                                values=['Бассейн'],
                                aggfunc=[len]
                                )
    if print_hist:
        df_pivot.hist()
    
    return df_pivot.value_counts()

In [ ]:
with Session(engine) as session:
    query = session.query(
                        PortCall.ship_id, 
                        Ship.type,
                        Port.basin,
                        func.count(PortCall.arrival).label('Количество судозаходов')
                        ).\
                    join(Port, PortCall.port_id == Port.id).\
                    join(Ship, PortCall.ship_id == Ship.id).\
                    group_by(PortCall.ship_id,  Ship.type, Port.basin)

    df = pd.read_sql(query.statement, query.session.bind)

ship_types = df['Тип судна'].drop_duplicates().dropna().to_list()
rows = {'Тип судна': [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: []}
for st in ship_types:
    r = various_bassin(df, ship_type=st)
    rows['Тип судна'].append(st)

    for i in range(6):
        try:
            rows[i+1].append(r[i+1])
        except:
            rows[i+1].append(np.NaN)

df_various_bassin= pd.DataFrame(rows)
df_various_bassin.to_excel('Вариативность_бассейнов_по_типам_судов.xlsx')

Проверка распределений возраста и тоннажности судов (рис 7 и 8)

In [ ]:
def age_histogram(date_from, date_to, port, ship_type):
    with Session(engine) as session:
        query = session.query((date.today().year - Ship.year).label('Возраст')).\
                        join(PortCall, PortCall.ship_id == Ship.id).\
                        join(Port, PortCall.port_id == Port.id).\
                        filter(Port.name.in_(tuple(port))).\
                        filter(Ship.type.in_(tuple(ship_type))).\
                        filter(Ship.year != None).\
                        filter(Ship.year != 0).\
                        filter(PortCall.ship_id != None).\
                        filter(PortCall.arrival >= date_from).\
                        filter(PortCall.arrival <= date_to)

        df = pd.read_sql(query.statement, query.session.bind)
    fig = go.Figure(go.Histogram(x=df['Возраст']))
    fig.update_layout(
        margin=dict(l=0, r=0, t=20, b=20),
        xaxis_title="возраст корабля",
        yaxis_title="частота заданного возраста", 
        )
    
    return fig, df

def tonnage_histogram(date_from, date_to, port, ship_type):
    with Session(engine) as session:
        query = session.query(Ship.tonnage).\
                        join(PortCall, PortCall.ship_id == Ship.id).\
                        join(Port, PortCall.port_id == Port.id).\
                        filter(Port.name ==port).\
                        filter(Ship.type.in_(tuple(ship_type))).\
                        filter(Ship.tonnage != None).\
                        filter(Ship.tonnage != 0).\
                        filter(PortCall.ship_id != None).\
                        filter(PortCall.arrival >= date_from).\
                        filter(PortCall.arrival <= date_to)

        df = pd.read_sql(query.statement, query.session.bind)

    fig = go.Figure(go.Histogram(x=df['Регист. вместим. валовая [т]']))
    fig.update_layout(
        margin=dict(l=0, r=0, t=20, b=20),
        xaxis_title="Регист. вместим. валовая [т]",
        yaxis_title="частота заданной вместимости", 
        )
    
    return fig, df

def get_best_distribution(data, print_info = False):
    dist_names = ["norm", "exponweib", "weibull_max", "weibull_min", "pareto", "genextreme"]
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        param = dist.fit(data)

        params[dist_name] = param
        D, p = st.kstest(data, dist_name, args=param)
        if print_info:
            print("p value for "+dist_name+" = "+str(p))
        dist_results.append((dist_name, p))

    # select the best fitted distribution
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    # store the name of the best fit and its p value

    if print_info:
        print("Best fitting distribution: "+str(best_dist))
        print("Best p value: "+ str(best_p))
        print("Parameters for the best fit: "+ str(params[best_dist]))

    return best_dist, best_p, params[best_dist]

In [ ]:
with Session(engine) as session:
        query = session.query(Port.name, Port.basin)
        port_df = pd.read_sql(query.statement, query.session.bind)


In [ ]:
# Возраст
port_df['best_distribution'] = '-'
port_df['p'] = '-'
port_df['params_best_dist'] = '-'

date_from=date(2021, 1, 1) 
date_to=date(2021, 12, 31)
ship_type=['60 / General cargo/multi-purpose ship']
for port_name in port_df['Название порта']:
    fig, df = age_histogram(date_from, date_to, [port_name], ship_type)
    if len(df)>10:
        best_dist, best_p, params_best_dist = get_best_distribution(df['Возраст'])
        port_df.loc[port_df['Название порта'] == port_name, 'best_distribution'] = best_dist
        port_df.loc[port_df['Название порта'] == port_name, 'p'] = best_p
        port_df.loc[port_df['Название порта'] == port_name, 'params_best_dist'] = '(' +', '.join(map(str, np.round(params_best_dist, 2))) + ')'

a = port_df[(port_df['best_distribution'] !='-')]
a[a['p']>0.01].sort_values(['Бассейн', 'best_distribution', 'p'])

In [ ]:
# Тоннаж
port_df['best_distribution'] = '-'
port_df['p'] = '-'
port_df['params_best_dist'] = '-'

date_from=date(2021, 1, 1) 
date_to=date(2021, 12, 31)
ship_type=['60 / General cargo/multi-purpose ship']
for port_name in port_df['Название порта']:
    fig, df = tonnage_histogram(date_from, date_to, port_name, ship_type)
    if len(df)>10:
        best_dist, best_p, params_best_dist = get_best_distribution(df['Регист. вместим. валовая [т]'])
        port_df.loc[port_df['Название порта'] == port_name, 'best_distribution'] = best_dist
        port_df.loc[port_df['Название порта'] == port_name, 'p'] = best_p
        port_df.loc[port_df['Название порта'] == port_name, 'params_best_dist'] = '(' +', '.join(map(str, np.round(params_best_dist, 2))) + ')'

a = port_df[(port_df['best_distribution'] !='-')]
a[a['p']>0.01].sort_values(['Бассейн', 'best_distribution', 'p'])

Графики показывающий зависимость длины, ширины судна, а также его типа, а также графики зависимости времени пребывания от регистрируемой валовой вместимости (рис 9-12)

In [ ]:
port = 'Азов'
with Session(engine) as session:
        query = session.query(PortCall.arrival, PortCall.departure, Ship.tonnage, Ship.type, Ship.length, Ship.width).\
                        join(PortCall, PortCall.ship_id == Ship.id).\
                        join(Port, PortCall.port_id == Port.id).\
                        filter(Port.name == port).\
                        filter(PortCall.arrival != None).\
                        filter(PortCall.departure != None)
        df = pd.read_sql(query.statement, query.session.bind)
df=df.dropna()
df['Время пребывания [ч]'] = (df['Дата/время отхода'] - df['Дата/время захода']).dt.total_seconds() / 3600
df['Тип судна (кратко)'] = df['Тип судна'].apply(lambda x: x.split('/')[0].strip())

In [ ]:
fig = px.scatter(df, x='Регист. вместим. валовая [т]', y= 'Время пребывания [ч]', color='Тип судна (кратко)', size='Длина наибольшая [м]')
fig.show()

fig = px.scatter(df, x='Длина наибольшая [м]', y= 'Ширина наибольшая [м]', color='Тип судна (кратко)')
fig.show()

df_b = df[df['Тип судна (кратко)'].isin(['60', '13', '85', '14'])]
fig = px.scatter(df_b, x='Длина наибольшая [м]', y= 'Ширина наибольшая [м]', color='Тип судна (кратко)')
fig.show()

fig = px.scatter(df_b, x='Регист. вместим. валовая [т]',  y='Время пребывания [ч]', color='Тип судна (кратко)', size='Длина наибольшая [м]')
fig.show()

# Анализ портов как графа

Графическое представление системы портов. Ребра построены исходя из данных за период 01.10.2022-31.12.2022 по всем типам судов. Вес ребра изображен его толщиной. Цветом обозначены бассейны (рис 13)

In [ ]:
with Session(engine) as session:
        query = session.query(Port.name, Port.basin, Port.lat, Port.lon).\
                        order_by(Port.id)
        port_df = pd.read_sql(query.statement, query.session.bind)

In [ ]:
adj_table = np.zeros((107, 107), dtype=int) # порт отправления, порт прибытия
ships = dict() # корабль, нынешний порт

date_from=date(2022, 10, 1) 
date_to=date(2022, 12, 31)

with Session(engine) as session:
        query = session.query(PortCall.ship_id, PortCall.port_id).\
                        filter(PortCall.arrival >= date_from).\
                        filter(PortCall.arrival <= date_to).\
                        order_by(PortCall.arrival.asc())
        
        for v in query:
            if v[0] in ships.keys():
                adj_table[ships[v[0]], v[1]] += 1
            ships[v[0]] = v[1]
                        
np.sum(adj_table)

In [ ]:
G = nx.DiGraph()
nodes = [(node, {'name': attr['Название порта'], 'basin': attr['Бассейн']}) for (node, attr) in port_df.to_dict('index').items()]
G.add_nodes_from(nodes)

edges = []
for i in range(107):
    for j in range(107):
        if adj_table[i, j] !=0:
            G.add_edge(i, j, weight=adj_table[i, j])


nx.draw(G, node_size=50)

In [ ]:
fig = plt.figure(figsize=(12, 9))

m = Basemap(
    projection='merc', #mill merc
    llcrnrlat=30,
    urcrnrlat=85,
    llcrnrlon=-10,
    urcrnrlon=200,
    resolution='c',
    )

m.drawcoastlines()
m.drawcountries()
# m.fillcontinents(color='coral',lake_color='aqua')
# m.drawmapboundary(fill_color='aqua')

#зададим положение, цвет и подпись для вершины графа (порта)
basin_color = {
    'Азово-Черноморский бассейн': 'olivedrab', 
    'Дальневосточный бассейн': 'dodgerblue',
    'Балтийский бассейн': 'firebrick', 
    'Арктический бассейн': 'aquamarine', 
    'Каспийский бассейн': 'blueviolet', 
    'Беломорско-Онежский бассейн': 'orange', 
    'Другое': 'pink'}
pos = dict()
node_labels = dict()
node_colors = []
for index, row in port_df.iterrows():
    pos[index] = m(row['lon'], row['lat'])
    node_labels[index] = row['Название порта']
    node_colors.append(basin_color[row['Бассейн']])

nx.draw_networkx_nodes(G, pos, node_size=20, node_color=node_colors)
nx.draw_networkx_labels(G,pos, font_size=3, labels=node_labels, horizontalalignment='right', verticalalignment='top')

# зададим толщину ребер графа 
n_nodes = G.number_of_nodes()
all_weights = []
for (node1,node2,data) in G.edges(data=True):
        all_weights.append(data['weight']) 
unique_weights = list(set(all_weights))

for weight in unique_weights:
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in G.edges(data=True) if edge_attr['weight']==weight]
        width = np.log(weight)*n_nodes*10/sum(all_weights)
        nx.draw_networkx_edges(G,pos,edgelist=weighted_edges,width=width, arrowstyle='->')

plt.title('Морские порты Российской Федерации')
plt.show()

Результаты исследования основных характеристик сети портов (рис 14)

In [ ]:
def task_radius_diametr(G, n_of_nodes = 300):
    G = nx.Graph(G)
    max_component = list(max(nx.connected_components(G), key=len))
    max_component_len = len(max_component)

    # N случайных вершин
    max_component_cut = set()
    i=0
    while i < n_of_nodes and i < max_component_len:
        max_component_cut.add(choice(max_component))
        i = len(max_component_cut)
    max_component_cut = list(max_component_cut)

    distance = []      # Расстояние (геодезическое)
    eccentricity = []  # Эксцентриситет
    for u in max_component_cut:
        d = [nx.shortest_path_length(G, source=u, target=v) for v in max_component_cut] 
        eccentricity.append(max(d))
        distance += d

    radius = min(eccentricity)   # Радиус
    diameter = max(eccentricity) # Диаметр
    percentile = np.percentile(distance, 90) # 90 процентиль расстояния между вершинами графа (в 90% случаев расстояние между вершинами <= этому значению)

    print(f'Для наибольшей компоненты слабой связности оценены')
    print(f'Радиус: {radius}')
    print(f'Диаметр: {diameter}')
    print(f'90 процентиль расстояния между вершинами графа: {round(percentile, 2)}')

def task_triangles_clustering(G):

    G = nx.Graph(G) # приведен к неориентированному

    triangles = sum(nx.triangles(G).values())/3
    average_clustering = nx.average_clustering(G)
    transitivity = nx.transitivity(G)

    print(f'Число треугольников в сети: {round(triangles)}')
    print(f'Средний кластерный коэффииент: {round(average_clustering, 2)}')
    print(f'Глобальный кластерный коэфициент: {round(transitivity, 2)}')

In [ ]:
n_of_nodes = G.number_of_nodes()
n_of_edges = G.number_of_edges()
density =  np.mean([degree[1] for degree in G.degree()]) / (n_of_nodes - 1)


n_weak = nx.number_weakly_connected_components(G)
largest_len = len(max(nx.weakly_connected_components(G), key=len))

print(f'Число вершин: {n_of_nodes}')
print(f'Число рёбер: {n_of_edges}')
print(f'Плотность: {round(density, 4)}')
print(f'Число компонент слабой связности: {n_weak}')
print(f'Доля вершин в максимальной по мощности компоненте слабой связности: {round(largest_len / n_of_nodes, 2)}')
print(f'Число компонент сильной связности: {nx.number_strongly_connected_components(G)}')
print(f'Доля вершин в максимальной по мощности компоненте сильной связности: {round(len(max(nx.strongly_connected_components(G), key=len))/ n_of_nodes, 2)}')


deg_avg = np.mean([degree[1] for degree in G.degree()])
count = len([node for node in G.nodes if G.degree(node) > deg_avg])

print(f'Число вершин степени, превышающей среднюю степень ({round(deg_avg, 2)}): {count}')

print()
task_radius_diametr(G)

print()
task_triangles_clustering(G)

Графики плотности распределения (рис 15)

In [ ]:
def task_pdf(G, nbinsx = 20):

    node_degrees = [degree[1] for degree in G.degree()]

    
    fig = go.Figure(data=[go.Histogram(x=node_degrees, nbinsx=nbinsx)])
    fig.update_layout(
        title='PDF в обычных шкалах',
        xaxis_title_text='Степень узла',
        yaxis_title_text='Частота',
        bargap=0.2,
        bargroupgap=0.1,
        margin=dict(t=30, b=10, l=10, r=10, pad=0)
    )

    deg_max = np.max(node_degrees)
    X = np.arange(deg_max + 1)
    Y = [len([node for node in G.nodes if G.degree(node) == i]) for i in X]
    fig_log = go.Figure(go.Scatter(mode="markers", x=X, y=Y))
    fig_log.update_xaxes(type="log")
    fig_log.update_yaxes(type="log")
    fig_log.update_layout(
        title='PDF в log - log шкалах',
        xaxis_title_text='Степень узла',
        yaxis_title_text='Частота',
        bargap=0.2,
        bargroupgap=0.1,
        margin=dict(t=30, b=10, l=10, r=10, pad=0)
    )

    fig.show()
    fig_log.show()

task_pdf(G, None)

Доля вершин в максимальной по мощности компоненте слабой связности после удаления некоторого % узлов (таблица 4)

In [ ]:
def task_delate(G, x = 0.2):
    number_of_nodes = G.number_of_nodes()        
    G_deleted_n = round(number_of_nodes * x)

    print(f'1. Удалены случайным образом {round(x * 100, 2)}% узлов (кол-во: {G_deleted_n})')
    deleted_names = []
    node_names = [degree[0] for degree in G.degree()]

    while len(deleted_names) < G_deleted_n:
        deleted_n = randrange(len(node_names))
        deleted_names.append(node_names[deleted_n])
        node_names.pop(deleted_n)

    G_rand = G.copy()
    G_rand.remove_nodes_from(deleted_names)
    largest_len_rand = len(max(nx.weakly_connected_components(G_rand), key=len)) if nx.is_directed(G) else len(max(nx.connected_components(G_rand), key=len))
    print(f'Доля вершин в максимальной по мощности компоненте слабой связности: {round(largest_len_rand / G_rand.number_of_nodes(), 2)}')

    print(f'2. Удалены {round(x * 100, 1)}% узлов наибольшей степени (кол-во: {G_deleted_n})')
    nodes_sorted = sorted([degree for degree in G.degree()], key=lambda tup: tup[1], reverse=True)
    deleted_names = [nodes_sorted[i][0] for i in range(G_deleted_n)]

    G_sorted = G.copy()
    G_sorted.remove_nodes_from(deleted_names)
    largest_len_sorted = len(max(nx.weakly_connected_components(G_sorted), key=len)) if nx.is_directed(G) else len(max(nx.connected_components(G_sorted), key=len))
    print(f'Доля вершин в максимальной по мощности компоненте слабой связности: {round(largest_len_sorted / G_sorted.number_of_nodes(), 2)}')
for p in range(10):
    task_delate(G, p/10)
    print()

Метрики центральности (таблица 5)

In [ ]:
def task_metric_data(G):

    max_component = max(nx.weakly_connected_components(G), key=len)
    
    G_max_component = nx.Graph()            # граф макс компоненты слабой связности
    for node in max_component:              # добавим вершины
        G_max_component.add_node(node)
    for (i, j) in list(G.edges):            # добавляем вершины
        if i in max_component and j in max_component:
            G_max_component.add_edge(i, j)
    
    G = nx.Graph(G_max_component)           # наибольшая компонента слабой связности, принятая за неориентированный граф

    return G


def task_centrality(G, delta = 0.2):
    # https://arxiv.org/pdf/1608.05845.pdf -- формулы

    degree_centrality = nx.centrality.degree_centrality(G)
    closeness_centrality = nx.centrality.closeness_centrality(G)
    betweenness_centrality = nx.centrality.betweenness_centrality(G)
    eigenvector_centrality = nx.centrality.eigenvector_centrality(G)

    # decay centrality
    # http://papers.econ.ucy.ac.cy/RePEc/papers/04-16.pdf -- формула
    decay_centrality = {}
    for i in list(G.nodes()):
        decay_centrality[i] = 0
        for j in list(G.nodes()):
            if i != j:
                distance = nx.shortest_path_length(G, source = i, target = j)
                decay_centrality[i] += delta ** distance

    return degree_centrality, closeness_centrality, betweenness_centrality, eigenvector_centrality, decay_centrality

    
def task_pagerank_hits(G):

    pagerank = nx.pagerank(G)
    hubs, authorities = nx.hits(G)

    return pagerank, hubs, authorities

In [ ]:
G_m = task_metric_data(G)

delta = 0.2
degree_cent, closeness_cent, betweenness_cent, eigenvector_cent, decay_cent = task_centrality(G_m, delta=delta)

pagerank, hubs, authorities = task_pagerank_hits(G_m)

In [ ]:
metrics = {0: ['degree cenrality', degree_cent], 
           1: ['closeness cenrality', closeness_cent], 
           2: ['betweenness cenrality', betweenness_cent], 
           3: ['eigenvector cenrality', eigenvector_cent], 
           4: [f'decay cenrality (delta = {delta})', decay_cent, delta], 
           5: ['pagerank', pagerank], 
           6: ['hubs', hubs],
           7: ['authorities', authorities]
}

metric = metrics[6]

metrics_sorted = sorted(metric[1].items(), key=lambda item: item[1], reverse=True)
first10 = pd.DataFrame(metrics_sorted[:10], columns=['Порт', metric[0]])
first10[metric[0]] = round(first10[metric[0]], 4)
first10['Порт'] = first10.apply(lambda x: port_df.at[x['Порт'], 'Название порта'], axis=1)
print(first10)

In [ ]:
g_m = metric[1]
for i in (set(range(107)) - set(metric[1].keys())):
    g_m[i] = 0

In [ ]:
fig = plt.figure(figsize=(12, 9))

m = Basemap(
    projection='merc', #mill merc
    llcrnrlat=30,
    urcrnrlat=85,
    llcrnrlon=-10,
    urcrnrlon=200,
    resolution='c',
    )

m.drawcoastlines()
m.drawcountries()

#зададим положение, цвет и подпись для вершины графа (порта)
pos = dict()
node_labels = dict()
node_sizes = []
for index, row in port_df.iterrows():
    pos[index] = m(row['lon'], row['lat'])
    node_labels[index] = row['Название порта']


    node_sizes.append(g_m[index])

node_sizes = np.array(node_sizes)
node_sizes = (((node_sizes - node_sizes.min()) / (node_sizes.max() - node_sizes.min())) * 10) ** 3

colors = range(len(set(node_sizes)))
cmap = plt.cm.viridis #plt.cm.Blues
bounds = sorted(list(set(node_sizes)))
norm = mpl_colors.BoundaryNorm(bounds, cmap.N, extend='both')
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm._A = []
plt.colorbar(sm)

nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color = node_sizes, cmap=cmap)
nx.draw_networkx_labels(G,pos, font_size=3, labels=node_labels, horizontalalignment='right', verticalalignment='top')

# зададим толщину ребер графа 
n_nodes = G.number_of_nodes()
all_weights = []
for (node1,node2,data) in G.edges(data=True):
        all_weights.append(data['weight']) 
unique_weights = list(set(all_weights))

for weight in unique_weights:
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in G.edges(data=True) if edge_attr['weight']==weight]
        width = np.log(weight)*n_nodes*10/sum(all_weights)
        nx.draw_networkx_edges(G,pos,edgelist=weighted_edges,width=width, arrowstyle='->')

plt.title(f'Морские порты Российской Федерации ({metric[0]})')
plt.show()

Максимальная клика (полный подграф Kn)

In [ ]:
# Максимальная клика (полный подграф Kn). На графе выделены вершины, принадлежащие максимальной клике
def max_clique(G, port_df):
    G = nx.Graph(G)
    all_cliques = list(nx.find_cliques(G))
    max_clique = sorted(all_cliques, key=len, reverse=True)[0]
    max_clique_names = [f"{port_df.iloc[i]['Название порта']} ({port_df.iloc[i]['Бассейн']})" for i in max_clique]
    print(f'Максимальная клика (размер: {len(max_clique)}): {max_clique_names}')

    plt.figure(figsize=(12, 9))

    m = Basemap(
        projection='merc', #mill merc
        llcrnrlat=37,
        urcrnrlat=62,
        llcrnrlon=120,
        urcrnrlon=180,
        resolution='c',
        )

    m.drawcoastlines()
    m.drawcountries()

    #зададим положение, цвет и подпись для вершины графа (порта)
    horizontalalignment = {
        'Владивосток' : 'right',
        'Находка': 'left',
        'Ванино': 'center', 
        'Восточный':'left', 
        'Корсаков': 'center', 
        'Славянка': 'right',
        'Петропавловск-Камчатский': 'center',
        'Магадан': 'center'
        }

    verticalalignment = {
        'Владивосток' : 'bottom',
        'Находка': 'bottom',
        'Ванино': 'center', 
        'Восточный':'top', 
        'Корсаков': 'center', 
        'Славянка': 'center',
        'Петропавловск-Камчатский': 'center',
        'Магадан': 'center'
        }

    pos = dict()
    node_labels = dict()
    node_colors = ['olivedrab' if node in max_clique else 'aquamarine' for node in G.nodes()]
    node_sizes = [100 if node in max_clique else 0 for node in G.nodes()]
    font_sizes = dict(G.degree)
    for node in G.nodes():
        if node in max_clique:
            font_sizes[node] = 7
        else:
            font_sizes[node] = 0
    for index, row in port_df.iterrows():
        pos[index] = m(row['lon'], row['lat'])
        node_labels[index] = row['Название порта']
        
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors)
    for node, (x, y) in pos.items():
        if node in max_clique:
            plt.text(x, y, node_labels[node], 
                     fontsize=font_sizes[node], 
                     ha=horizontalalignment[node_labels[node]], 
                     va=verticalalignment[node_labels[node]], 
                     weight='bold',
                     bbox=dict(boxstyle="square",
                                ec=(1., 1.,  1.),
                                fc=(0.9, 0.9, 0.9),
                                alpha=0.5
                                )
                     )

    plt.title('Максимальная клика на графе морских портов РФ')
    plt.show()

max_clique(G, port_df)

Сообщества в графе (рис 16-17)

In [ ]:
def communities(G, partitions):
    print(f'Количество сообществ: {len(partitions)}')
    print(f'Значение метрики модулярности: {round(nx_comm.modularity(G, partitions), 4)}')

# максимизация метрики модулярности 
max_modularity_communities = nx_comm.greedy_modularity_communities(G_m)
communities(G_m, max_modularity_communities)

# на основе метрики edge betweeness
edge_betweenness = nx_comm.girvan_newman(G_m)
edge_betweenness = list(sorted(c) for c in next(edge_betweenness))
communities(G_m, edge_betweenness)


In [ ]:
color_comm = {0: 'olivedrab', 1: 'dodgerblue', 2: 'firebrick', 3: 'aquamarine', 4: 'blueviolet'}
comm_partition = edge_betweenness #edge_betweenness max_modularity_communities

fig = plt.figure(figsize=(12, 9))

m = Basemap(
    projection='merc', #mill merc
    llcrnrlat=30,
    urcrnrlat=85,
    llcrnrlon=-10,
    urcrnrlon=200,
    resolution='c',
    )

m.drawcoastlines()
m.drawcountries()
# m.fillcontinents(color='coral',lake_color='aqua')
# m.drawmapboundary(fill_color='aqua')

#зададим положение, цвет и подпись для вершины графа (порта)
pos = dict()
node_labels = dict()
node_colors = []
for index, row in port_df.iterrows():
    pos[index] = m(row['lon'], row['lat'])
    node_labels[index] = row['Название порта']

    comm_flag = False
    for i_comm in range(len(comm_partition)):
        if index in comm_partition[i_comm]:
            node_colors.append(color_comm[i_comm])
            comm_flag = True
            break
    if not comm_flag:
        node_colors.append(color_comm[4])
    
nx.draw_networkx_nodes(G, pos, node_size=20, node_color=node_colors)
nx.draw_networkx_labels(G,pos, font_size=3, labels=node_labels, horizontalalignment='right', verticalalignment='top')

# зададим толщину ребер графа 
n_nodes = G.number_of_nodes()
all_weights = []
for (node1,node2,data) in G.edges(data=True):
        all_weights.append(data['weight']) 
unique_weights = list(set(all_weights))

for weight in unique_weights:
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in G.edges(data=True) if edge_attr['weight']==weight]
        width = np.log(weight)*n_nodes*10/sum(all_weights)
        nx.draw_networkx_edges(G,pos,edgelist=weighted_edges,width=width, arrowstyle='->')

plt.title('Морские порты Российской Федерации')
plt.show()

# Временные ряды

In [ ]:
with Session(engine) as session:
        query = session.query(Port.name, Port.basin, Port.lat, Port.lon).\
                        order_by(Port.id)
        port_df = pd.read_sql(query.statement, query.session.bind)

def arrival_dynamics_with_tonnage(ports, ship_type, date_from, date_to):
        with Session(engine) as session:
                query = session.query(
                                func.date_trunc('day', PortCall.arrival).label('arrival_day'), 
                                func.sum(Ship.tonnage).label('tonnage')).\
                                join(PortCall, PortCall.ship_id == Ship.id).\
                                join(Port, PortCall.port_id == Port.id).\
                                filter(PortCall.arrival >= date_from).\
                                filter(PortCall.arrival <= date_to).\
                                filter(Port.name.in_(tuple(ports))).\
                                filter(Ship.type == ship_type).\
                                filter(PortCall.arrival != None).\
                                filter(PortCall.departure != None).\
                                filter(Ship.tonnage != np.NaN).\
                                group_by('arrival_day').\
                                order_by('arrival_day')
                
                df = pd.read_sql(query.statement, query.session.bind)

        days = set(df['arrival_day'].to_list())
        datelist = set(pd.date_range(min(days), max(days)).tolist())
        for d in (datelist-days):
                df.loc[len(df.index)] = [d, 0] 
        
        return df.sort_values('arrival_day').set_index('arrival_day')

In [ ]:
ports = port_df.loc[port_df['Бассейн'] == 'Балтийский бассейн', 'Название порта'].to_list()
with Session(engine) as session:
    query = session.query(
                    func.sum(Ship.tonnage).label('tonnage'),
                    PortCall.port_call).\
                    join(PortCall, PortCall.ship_id == Ship.id).\
                    join(Port, PortCall.port_id == Port.id).\
                    filter(Port.name.in_(tuple(ports))).\
                    filter(Ship.tonnage != np.NaN).\
                    group_by(PortCall.port_call).\
                    order_by('tonnage')
    port_tonnage = pd.read_sql(query.statement, query.session.bind)

port_tonnage[::-1]

In [ ]:
ports = port_df.loc[port_df['Бассейн'] == 'Азово-Черноморский бассейн', 'Название порта'].to_list()
with Session(engine) as session:
    query = session.query(
                    func.count(PortCall.ship_id).label('count'),
                    Ship.type).\
                    join(PortCall, PortCall.ship_id == Ship.id).\
                    join(Port, PortCall.port_id == Port.id).\
                    filter(Port.name.in_(tuple(ports))).\
                    filter(Ship.tonnage != np.NaN).\
                    group_by(Ship.type).\
                    order_by('count')
    ship_type_count = pd.read_sql(query.statement, query.session.bind)

ship_type_count[::-1]

In [ ]:
data_slice = {
    1: {'ports': port_df.loc[port_df['Бассейн'] == 'Азово-Черноморский бассейн', 'Название порта'].to_list(),
        'ship_type': '60 / General cargo/multi-purpose ship',
        'name': 'Азово-Черноморский бассейн'},
    2: {'ports': port_df.loc[port_df['Бассейн'] == 'Дальневосточный бассейн', 'Название порта'].to_list(),
        'ship_type': '13 / Oil tanker',
        'name': 'Дальневосточный бассейн'},
    3: {'ports': port_df.loc[port_df['Бассейн'] == 'Балтийский бассейн', 'Название порта'].to_list(),
        'ship_type': '60 / General cargo/multi-purpose ship',
        'name': 'Балтийский бассейн'},
    4: {'ports': ['Новороссийск'],
        'ship_type': '13 / Oil tanker',
        'name': 'Новороссийск'},
    5: {'ports': ['Восточный'],
        'ship_type': '13 / Oil tanker',
        'name': 'Восточный'},
    6: {'ports': ['Санкт-Петербург'],
        'ship_type': '60 / General cargo/multi-purpose ship',
        'name': 'Санкт-Петербург'}
}

for i_data_slice in range(1, 7):
    date_from=date(2015, 1, 1) 
    date_to=date(2023, 12, 31)
    ports = data_slice[i_data_slice]['ports']
    ship_type = data_slice[i_data_slice]['ship_type']
    df = arrival_dynamics_with_tonnage(ports, ship_type, date_from, date_to)
    df_min, df_max = df.min(), df.max()
    df = (df-df_min)/(df_max -df_min)
    df = df.resample('D').sum()
    train_start = date(2020, 12, 31)
    data_slice[i_data_slice]['train'] = df.loc[:train_start]
    data_slice[i_data_slice]['test']  = df.loc[train_start:]
    data_slice[i_data_slice]['all_data']  = df.copy()

Исходные временные ряды (рис 18)

In [ ]:
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles = [data_slice[(i % 2) * 3 + i // 2 + 1]['name'] for i in range(6)],
                    vertical_spacing = 0.1, horizontal_spacing=0.05)
for i in range(6):
    i_data_slice = i + 1
    row = i % 3 + 1
    col = i // 3 + 1
    fig.add_trace(go.Scatter(
        x=data_slice[i_data_slice]['train'].index, y=data_slice[i_data_slice]['train'].tonnage, 
        name='Train', line=dict(color='MediumPurple'), showlegend=False), row=row, col=col)
    fig.add_trace(go.Scatter(
        x=data_slice[i_data_slice]['test'].index, y=data_slice[i_data_slice]['test'].tonnage, 
        name='Test', line=dict(color='Olive'), showlegend=False), row=row, col=col)
fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
fig.update_annotations(font_size=12)

fig.show()

Значения периода методом периодограмм (таблица 6)

In [ ]:
def period_calculation(data):

    # Вычисление периодограммы
    frequencies, power_spectrum = periodogram(data.tonnage)

    # Нахождение наибольшей мощности
    max_power_idx = np.argmax(power_spectrum)
    dominant_frequency = frequencies[max_power_idx]

    # Определение периода
    period = 1 / dominant_frequency
    return period

for i in range(1, 7):
    print(data_slice[i]["name"],':', period_calculation(data_slice[i]["train"]))

Разложение временного ряда на составляюще методом STL (рис 19)

In [ ]:
def model_STL(data_slice, period=365, threshold=3, forecast_period=365, resample = 'D', stl_train=True, arima_args=dict(order=(1, 1, 0), trend="t")):
    model = dict()
    for i in range(1, 7):
        model[i] = dict()
        if stl_train:
            data = data_slice[i]['train'].tonnage.resample(resample).mean()
        else:
             data = data_slice[i]['all_data'].tonnage.resample(resample).mean()
        stl = STL(data, period=period)
        res = stl.fit()

        model[i]['trend'] = res.trend
        model[i]['seasonal'] = res.seasonal
        model[i]['residual'] = res.resid

        # Выявление аномалий
        model[i]['threshold'] = threshold * np.std(res.resid)

        #прогноз
        stlf = STLForecast(data, ARIMA, model_kwargs=arima_args)
        stlf_res = stlf.fit()

        model[i]['forecast'] = stlf_res.forecast(forecast_period)

    return model

In [ ]:
mstl = model_STL(data_slice, period=52, threshold=3,forecast_period=130,  resample='W') # 52, 130, 'W'

In [ ]:
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles = [data_slice[(i % 2) * 3 + i // 2 + 1]['name'] for i in range(6)],
                    vertical_spacing = 0.1, horizontal_spacing=0.05)
for i in range(6):
    i_data_slice = i + 1
    row = i % 3 + 1
    col = i // 3 + 1
    fig.add_trace(go.Scatter(
        x=data_slice[i_data_slice]['train'].index, y=data_slice[i_data_slice]['train'].tonnage, 
        name='Train', line=dict(color='MediumPurple'), showlegend=False), row=row, col=col)
    fig.add_trace(go.Scatter(
        x=mstl[i_data_slice]['trend'].index, y=mstl[i_data_slice]['trend'], 
        name='Train', line=dict(color='navy'), showlegend=False), row=row, col=col)
    fig.add_trace(go.Scatter(
        x=mstl[i_data_slice]['seasonal'].index, y=mstl[i_data_slice]['seasonal'], 
        name='Train', line=dict(color='navy'), showlegend=False), row=row, col=col)
    
fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
fig.update_annotations(font_size=12)

fig.show()

Данные для описания тренда (Таблица 7) 

In [ ]:
for i in range(1, 7):
    print(mstl[i]['trend'].describe())
    print(mstl[i]['trend'].max() - mstl[i]['trend'].min())
    print(mstl[i]['trend'][-1] - mstl[i]['trend'][0])

Прогноз с помощью STLForecast (рис 20)

In [ ]:
def plot_STL(data_slice, model_stl, train=True, test=True):
    fig = make_subplots(rows=3, cols=2, 
                        subplot_titles = [data_slice[(i % 2) * 3 + i // 2 + 1]['name'] for i in range(6)],
                        vertical_spacing = 0.1, horizontal_spacing=0.05)
    for i in range(6):
        i_data_slice = i + 1
        row = i % 3 + 1
        col = i // 3 + 1
        if train:
            fig.add_trace(go.Scatter(
                x=data_slice[i_data_slice]['train'].index, y=data_slice[i_data_slice]['train'].tonnage, 
                name='Train', line=dict(color='MediumPurple'), showlegend=False), row=row, col=col)
            fig.add_trace(go.Scatter(
                x=model_stl[i_data_slice]['trend'].index, y=model_stl[i_data_slice]['trend']+model_stl[i_data_slice]['seasonal'], 
                name='Train', line=dict(color='navy'), showlegend=False), row=row, col=col)
        if test:
            y = data_slice[i_data_slice]['test'].tonnage.resample('W').mean()

            fig.add_trace(go.Scatter(
                x=y.index, y=y, 
                name='Test', line=dict(color='Olive'), showlegend=False), row=row, col=col)
            fig.add_trace(go.Scatter(
                x=model_stl[i_data_slice]['forecast'][:pd.Timestamp('2023-04-02')].index, y=model_stl[i_data_slice]['forecast'][:pd.Timestamp('2023-04-02')], 
                name='Train', line=dict(color='mediumvioletred'), showlegend=False), row=row, col=col)
    fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
    fig.update_annotations(font_size=12)

    fig.show()
    
plot_STL(data_slice, mstl, train=False, test=True)

Прогноз с помощью LSTM (рис 21)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, lookback):
        super().__init__()
        self.lstm = nn.LSTM(input_size=lookback, hidden_size=64, num_layers=1, batch_first=True).type(torch.float64)
        self.linear = nn.Linear(64, lookback).type(torch.float64)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x
    
def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(len(dataset)-2*lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+lookback+1:i+2*lookback+1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

In [ ]:
def model_lstm(X_train, y_train, X_test, y_test, lookback, n_epochs = 1000, print_bool=True):
    model = LSTM(lookback)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = nn.MSELoss()
    loader = DataLoader(TensorDataset(X_train, y_train), shuffle=True, batch_size=8)
    
    for epoch in range(n_epochs):
        model.train()
        for X_batch, y_batch in loader:
            X_batch = X_batch.type(torch.float64)
            y_batch = y_batch.type(torch.float64)
            y_pred = model(X_batch)
            # print(y_pred.dtype)
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # Validation
        if (epoch % 100 != 0) and print_bool:
            continue
        model.eval()
        with torch.no_grad():
            y_pred = model(X_train)
            train_rmse = np.sqrt(loss_fn(y_pred, y_train))
            y_pred = model(X_test)
            test_rmse = np.sqrt(loss_fn(y_pred, y_test))
        print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

    return model

In [ ]:
res = dict()
for j in range(1, 7):
    print()
    print(data_slice[j]['name'])
    res[j] = dict()
    res[j]['name'] = data_slice[j]['name']
    
    #условия
    date_from=date(2015, 1, 1) 
    date_to=date(2023, 12, 31)
    ports = data_slice[j]['ports']
    ship_type = data_slice[j]['ship_type']

    #изначальный датасет
    df = arrival_dynamics_with_tonnage(ports, ship_type, date_from, date_to).tonnage
    df_min, df_max = df.min(), df.max()
    df = (df-df_min)/(df_max -df_min)
    df = df.resample('W').mean()

    # датасет разбитый на train и test
    lookback = 52
    train_border = date(2020, 12, 31)
    train, test = df.loc[:train_border], df.loc[train_border - timedelta(lookback*7):]
    X_train, y_train = create_dataset(train, lookback=lookback)
    X_test, y_test = create_dataset(test, lookback=lookback)

    # обучение
    model = model_lstm(X_train, y_train, X_test, y_test, lookback, n_epochs = 300, print_bool=True)

    # прогноз
    forecast_df = pd.Series([0]*len(test), index=test.index)
    input = X_test[0]
    forecast_df[:lookback] = pd.Series(X_test[0], index=test[:lookback].index)
    without_pred = len(forecast_df) - lookback

    for i in range(len(forecast_df) - lookback):
        input = forecast_df[i:i+lookback]
        input = torch.tensor(input).unsqueeze(0).type(torch.float64)
        
        if without_pred >= lookback:
            forecast_df[i+lookback: i+2*lookback] = pd.Series(model(input)[0].detach().numpy(), index=test[i+lookback: i+2*lookback].index)
        else:
            pred = model(input)[0].detach().numpy()[:without_pred]
            forecast_df[i+lookback: i+lookback+without_pred] = pd.Series(pred, index=test[i+lookback: i+lookback+without_pred].index)
        without_pred -=1

    forecast_df = forecast_df.loc[train_border:]
    test = test[train_border:]
    res[j]['forecast'] = forecast_df
    res[j]['test'] = test

In [ ]:
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles = [data_slice[(i % 2) * 3 + i // 2 + 1]['name'] for i in range(6)],
                    vertical_spacing = 0.1, horizontal_spacing=0.05)
for i in range(6):
    fig.add_trace(go.Scatter(x=res[i + 1]['forecast'].index, y=res[i + 1]['forecast'], name='prediction',
                             line=dict(color='mediumvioletred'), showlegend=False), row=i%3+1, col=i//3+1)
    fig.add_trace(go.Scatter(x=res[i + 1]['test'].index, y=res[i + 1]['test'], name='ground truth',
                             line=dict(color='Olive'), showlegend=False), row=i%3+1, col=i//3+1)
    
fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
fig.update_annotations(font_size=12)

fig.show()

t-критерий (рис 22 и 24)

In [ ]:
events_df = pd.read_excel('Приложения/external_events_15_22.xlsx', parse_dates=True)
events_df['Average_date'] = events_df.apply(lambda x: x['Дата начала'] + (x['Дата начала'] - x['Дата окончания'])/2, axis=1)

events_df.loc[(events_df['География влияния'] =='Порт') | (events_df['География влияния'] =='Бассейн'), 'География влияния'] = events_df['Подробнее']
events_df = events_df.drop(columns='Подробнее')

influence_rows = []

for basin in port_df['Бассейн'].drop_duplicates():
    influence_rows.append([basin, basin, 'bb'])
    influence_rows.append([basin, 'Россия', 'brf'])
for port in port_df['Название порта']:
    influence_rows.append([port, port, 'p'])
    influence_rows.append([port, 'Россия', 'prf'])  
    basin = port_df.loc[port_df['Название порта'] == port, 'Бассейн'].to_list()[0]
    influence_rows.append([port, basin, 'pb'])  
    influence_rows.append([basin,port, 'bp'])
    influence_rows += [[port, p, 'pbp'] for p in port_df.loc[port_df['Бассейн'] == basin, 'Название порта'].to_list() if port !=p] 

dependency_df = pd.DataFrame(influence_rows, columns=['Influenced', 'Influencer', 'Influence_type'])

In [ ]:
significance_df = events_df.copy()
l = 1 # рассматриваемая локация
residual = STL(data_slice[l]['all_data'].tonnage, period=365).fit().resid
periods = [7, 30, 30*3, 30*6, 365]
for p in periods:
    significance_df[f'h={p}, before mean'] = np.NaN
    significance_df[f'h={p}, before std'] = np.NaN
    significance_df[f'h={p}, after mean'] = np.NaN
    significance_df[f'h={p}, after std'] = np.NaN
    significance_df[f'h={p}, t-statistic'] = np.NaN
    significance_df[f'h={p}, p-value'] = np.NaN

Influenced = data_slice[l]['name']
Influencer = dependency_df.loc[dependency_df['Influenced'] == Influenced, 'Influencer'].to_list()
ev = events_df.loc[events_df['География влияния'].isin(Influencer)]

for i, row in ev.iterrows():
    event_date = row['Average_date']
    for p in periods:
        before_event = residual.loc[(residual.index < pd.to_datetime(event_date)) & (residual.index >= event_date-timedelta(days=p))]
        after_event = residual.loc[(residual.index >= pd.to_datetime(event_date)) & (residual.index <= event_date+timedelta(days=p))]

        t_stat, p_value = ttest_ind(before_event, after_event)

        significance_df.at[i, f'h={p}, before mean'] = before_event.mean()
        significance_df.at[i, f'h={p}, before std'] =  after_event.mean()
        significance_df.at[i, f'h={p}, after mean'] = before_event.std()
        significance_df.at[i, f'h={p}, after std'] = after_event.std()
        significance_df.at[i, f'h={p}, t-statistic'] = t_stat
        significance_df.at[i, f'h={p}, p-value'] = p_value

significance_df.loc[(abs(significance_df['h=30, t-statistic'])>=1.96) & (significance_df['h=30, p-value'] < 0.05), ['Average_date', 'География влияния', 'Описание', 'Влияние', 'h=30, t-statistic', 'h=30, p-value']]

In [ ]:
h_df = significance_df.copy()
h_df['max_d'] = 0
h_df['diff'] = np.NaN
for i, h in enumerate(periods):
    h_df[h] = h_df.apply(lambda x: x[f'h={h}, before mean']-x[f'h={h}, after mean'] if abs(x[f'h={h}, t-statistic'])>1.96 and x[f'h={h}, p-value']<0.05 else np.NaN, axis=1)
    h_df['max_d'] = h_df.apply(lambda x: h if not pd.isna(x[h]) else x['max_d'], axis=1)
    h_df['diff'] = h_df.apply(lambda x: x[h] if not pd.isna(x[h]) else x['diff'], axis=1)
h_df = h_df.dropna(subset='diff')
h_df['Влияние'] = h_df['Влияние'].astype('int').astype('str')

pt = pd.pivot_table(h_df, index='Влияние', values=['diff', 'max_d'], aggfunc=['mean', 'std', 'min', 'max'], dropna=False, fill_value=0)
pt

Рисунок представляет собой шум от временного ряда для Азово-Черноморского бассейна после разложения STL c наложением значимых событий (рис 23)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=residual.index, y=residual))
h_df = h_df.set_index('Average_date')
for i, r in h_df.loc[(h_df['h=30, p-value'] < 0.05)].iterrows():
    fig.add_shape(
                type='line',
                line_color='gold',
                line_width=3,
                opacity=.5,
                x0=i,
                x1=i,
                xref='x',
                y0=residual.min()*1.1,
                y1=residual.max()*1.1,
                yref='y',
                )
fig.show()

Моделирование временного ряда с помощью STL и добавлением экзогенного параметра (рис 25)
Также подсчитаны MAE, MSE, MAPE для STL и STLexog (таблицы 8-10)

In [ ]:
traces = dict()
arima_args = {1: dict(order=(1, 1, 0), trend="t"), 2: dict(order=(1, 0, 0)), 3: dict(order=(1, 1, 0)),
               4: dict(order=(1, 1, 0)), 5: dict(order=(1, 1, 0)), 6:dict(order=(1, 1, 1))}
coeff = {1: 2, 2: .5, 3: 1, 4: 1, 5: .25, 6: 1}

for l in range(1, 7): # рассматриваемая локация
    print(data_slice[l]['name'])
    significance_df = events_df.copy()
    residual = STL(data_slice[l]['all_data'].tonnage, period=365).fit().resid
    periods = [7, 30, 30*3, 30*6, 365]
    for p in periods:
        significance_df[f'h={p}, before mean'] = np.NaN
        significance_df[f'h={p}, before std'] = np.NaN
        significance_df[f'h={p}, after mean'] = np.NaN
        significance_df[f'h={p}, after std'] = np.NaN
        significance_df[f'h={p}, t-statistic'] = np.NaN
        significance_df[f'h={p}, p-value'] = np.NaN

    Influenced = data_slice[l]['name']
    Influencer = dependency_df.loc[dependency_df['Influenced'] == Influenced, 'Influencer'].to_list()
    ev = events_df.loc[events_df['География влияния'].isin(Influencer)]

    for i, row in ev.iterrows():
        event_date = row['Average_date']
        for p in periods:
            before_event = residual.loc[(residual.index < pd.to_datetime(event_date)) & (residual.index >= event_date-timedelta(days=p))]
            after_event = residual.loc[(residual.index >= pd.to_datetime(event_date)) & (residual.index <= event_date+timedelta(days=p))]

            t_stat, p_value = ttest_ind(before_event, after_event)

            significance_df.at[i, f'h={p}, before mean'] = before_event.mean()
            significance_df.at[i, f'h={p}, before std'] =  after_event.mean()
            significance_df.at[i, f'h={p}, after mean'] = before_event.std()
            significance_df.at[i, f'h={p}, after std'] = after_event.std()
            significance_df.at[i, f'h={p}, t-statistic'] = t_stat
            significance_df.at[i, f'h={p}, p-value'] = p_value

    h_df = significance_df.copy()
    h_df['max_d'] = 0
    h_df['diff'] = np.NaN
    for i, h in enumerate(periods):
        h_df[h] = h_df.apply(lambda x: x[f'h={h}, before mean']-x[f'h={h}, after mean'] if abs(x[f'h={h}, t-statistic'])>1.96 and x[f'h={h}, p-value']<0.05 else np.NaN, axis=1)
        h_df['max_d'] = h_df.apply(lambda x: h if not pd.isna(x[h]) else x['max_d'], axis=1)
        h_df['diff'] = h_df.apply(lambda x: x[h] if not pd.isna(x[h]) else x['diff'], axis=1)
    h_df = h_df.dropna(subset='diff')
    h_df['Влияние'] = h_df['Влияние'].astype('int').astype('str')

    pt = pd.pivot_table(h_df, index='Влияние', values=['diff', 'max_d'], aggfunc=['mean', 'std', 'min', 'max'], dropna=False, fill_value=0)
    #print(pt)

    stlf = STLForecast(data_slice[l]['train'].tonnage.resample('W').mean(), ARIMA, model_kwargs=arima_args[l]).fit()
    test_prediction =stlf.forecast(117)
    test_data = data_slice[l]['test'].tonnage.resample('W').mean()[1:]

    a = h_df.copy().set_index('Average_date')
    a['Влияние'] = a['Влияние'].astype('int').astype('str')

    biased_correction = list()
    n = 500
    d_max = test_prediction.index.max()

    for i in range(n):
        biased_correction_i = test_prediction.copy()
        for j, r in a.iterrows():
            
            diff = np.random.normal(pt.at[r['Влияние'], ('mean', 'diff')], pt.at[r['Влияние'], ('std', 'diff')])
            t = np.random.normal(pt.at[r['Влияние'], ('mean', 'max_d')], pt.at[r['Влияние'], ('std', 'max_d')])

            t = min(j+timedelta(max(t, 0)), d_max)
            biased_correction_i[j:t] = diff * coeff[l]
        biased_correction_i +=test_prediction
        biased_correction.append(biased_correction_i)

    bc = test_prediction.copy()
    for i, d in enumerate(test_prediction.index):
        bc[d] = 0
        for j in range(n):
            bc[d] += biased_correction[j][d]
        bc[d] /= n

    mae_biased = mean_absolute_error(test_data, bc)
    mae_test = mean_absolute_error(test_data, test_prediction)
    print("MAE для прогноза на основе STL: ", mae_test)
    print("MAE для прогноза на основе метода смещенной корректировки: ", mae_biased)
    mse_biased = mean_squared_error(test_data, bc)
    mse_test = mean_squared_error(test_data, test_prediction)
    print("MSE для прогноза на основе STL: ", mse_test)
    print("MSE для прогноза на основе метода смещенной корректировки: ", mse_biased)
    mape_biased = mean_absolute_percentage_error(test_data, bc)
    mape_test = mean_absolute_percentage_error(test_data, test_prediction)
    print("MAPE для прогноза на основе STL: ", mape_test)
    print("MAPE для прогноза на основе метода смещенной корректировки: ", mape_biased)



    traces[l] = dict()
    traces[l]['Prediction'] = go.Scatter(x=test_prediction.index, y=test_prediction, name='Prediction', showlegend=False, line=dict(color='mediumvioletred'))
    traces[l]['Biased correction'] = go.Scatter(x=bc.index, y=bc, name='Biased correction', showlegend=False, line=dict(color='blue'))
    traces[l]['Ground truth'] = go.Scatter(x=test_data.index, y=test_data, name='Ground truth', showlegend=False, line=dict(color='Olive'))


In [ ]:
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles = [data_slice[(i % 2) * 3 + i // 2 + 1]['name'] for i in range(6)],
                    vertical_spacing = 0.1, horizontal_spacing=0.05)
for i in range(6):
    fig.add_trace(traces[i+1]['Ground truth'], row=i%3+1, col=i//3+1)
    fig.add_trace(traces[i+1]['Prediction'], row=i%3+1, col=i//3+1)
    fig.add_trace(traces[i+1]['Biased correction'], row=i%3+1, col=i//3+1)
    
fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
fig.update_annotations(font_size=12)

fig.show()

Моделирование временного ряда с помощью LSTM и добавлением экзогенного параметра (рис 26)
Также подсчитаны MAE, MSE, MAPE для LSTM и LSTMexog (таблицы 8-10)

In [ ]:
models = dict()
for j in range(1, 7):
    print()
    print(data_slice[j]['name'])
    
    #условия
    date_from=date(2015, 1, 1) 
    date_to=date(2023, 12, 31)
    ports = data_slice[j]['ports']
    ship_type = data_slice[j]['ship_type']

    #изначальный датасет
    df = arrival_dynamics_with_tonnage(ports, ship_type, date_from, date_to).tonnage
    df_min, df_max = df.min(), df.max()
    df = (df-df_min)/(df_max -df_min)
    df = df.resample('W').mean()

    # датасет разбитый на train и test
    lookback = 52
    train_border = date(2020, 12, 31)
    train, test = df.loc[:train_border], df.loc[train_border - timedelta(lookback*7):]
    X_train, y_train = create_dataset(train, lookback=lookback)
    X_test, y_test = create_dataset(test, lookback=lookback)

    # обучение
    models[j] = dict()
    models[j]['model'] = model_lstm(X_train, y_train, X_test, y_test, lookback, n_epochs = 500, print_bool=True)
    models[j]['train'] = train
    models[j]['test'] = test

In [ ]:
arima_args = {1: dict(order=(1, 1, 0), trend="t"), 2: dict(order=(1, 0, 0)), 3: dict(order=(1, 1, 0)),
               4: dict(order=(1, 1, 0)), 5: dict(order=(1, 1, 0)), 6:dict(order=(1, 1, 1))}
coeff = {1: 2, 2: .5, 3: 1, 4: 1, 5: .25, 6: 1}
pt = dict()
seg = dict()

for l in range(1, 7): # рассматриваемая локация
    significance_df = events_df.copy()
    residual = STL(data_slice[l]['all_data'].tonnage, period=365).fit().resid
    periods = [7, 30, 30*3, 30*6, 365]
    for p in periods:
        significance_df[f'h={p}, before mean'] = np.NaN
        significance_df[f'h={p}, before std'] = np.NaN
        significance_df[f'h={p}, after mean'] = np.NaN
        significance_df[f'h={p}, after std'] = np.NaN
        significance_df[f'h={p}, t-statistic'] = np.NaN
        significance_df[f'h={p}, p-value'] = np.NaN

    Influenced = data_slice[l]['name']
    Influencer = dependency_df.loc[dependency_df['Influenced'] == Influenced, 'Influencer'].to_list()
    ev = events_df.loc[events_df['География влияния'].isin(Influencer)]

    for i, row in ev.iterrows():
        event_date = row['Average_date']
        for p in periods:
            before_event = residual.loc[(residual.index < pd.to_datetime(event_date)) & (residual.index >= event_date-timedelta(days=p))]
            after_event = residual.loc[(residual.index >= pd.to_datetime(event_date)) & (residual.index <= event_date+timedelta(days=p))]

            t_stat, p_value = ttest_ind(before_event, after_event)

            significance_df.at[i, f'h={p}, before mean'] = before_event.mean()
            significance_df.at[i, f'h={p}, before std'] =  after_event.mean()
            significance_df.at[i, f'h={p}, after mean'] = before_event.std()
            significance_df.at[i, f'h={p}, after std'] = after_event.std()
            significance_df.at[i, f'h={p}, t-statistic'] = t_stat
            significance_df.at[i, f'h={p}, p-value'] = p_value

    h_df = significance_df.copy()
    h_df['max_d'] = 0
    h_df['diff'] = np.NaN
    for i, h in enumerate(periods):
        h_df[h] = h_df.apply(lambda x: x[f'h={h}, before mean']-x[f'h={h}, after mean'] if abs(x[f'h={h}, t-statistic'])>1.96 and x[f'h={h}, p-value']<0.05 else np.NaN, axis=1)
        h_df['max_d'] = h_df.apply(lambda x: h if not pd.isna(x[h]) else x['max_d'], axis=1)
        h_df['diff'] = h_df.apply(lambda x: x[h] if not pd.isna(x[h]) else x['diff'], axis=1)
    h_df = h_df.dropna(subset='diff')
    h_df['Влияние'] = h_df['Влияние'].astype('int').astype('str')

    seg[l] = h_df.set_index('Average_date')
    pt[l] = pd.pivot_table(h_df, index='Влияние', values=['diff', 'max_d'], aggfunc=['mean', 'std', 'min', 'max'], dropna=False, fill_value=0)

In [ ]:
res = dict()
steps = {1: 52, 2: 52, 3: 13, 4: 2, 5: 2, 6: 26}
coeff = {1: 1/64, 2: 1/16, 3: 1/4, 4: 1/4, 5: 1/4, 6: 1/8}
for j in range(1, 7):
    print()
    print(data_slice[j]['name'])

    # обучение
    model = models[j]['model']
    test  = models[j]['test']
    train = models[j]['train']

    n = 100
    forecast_df_list = list()
    for k in tqdm(range(n)):
        # прогноз
        forecast_df = pd.Series([0]*len(test), index=test.index)
        input = X_test[0]
        forecast_df[:lookback] = pd.Series(X_test[0], index=test[:lookback].index)
        without_pred = len(forecast_df) - lookback


        # прогноз
        forecast_df = pd.Series([0]*len(test), index=test.index)
        input = X_test[0]
        forecast_df[:lookback] = pd.Series(X_test[0], index=test[:lookback].index)
        
        without_pred = len(forecast_df) - lookback
        step = steps[j]
        
        for i in range(0, len(forecast_df) - lookback, step):
            input = forecast_df[i:i+lookback]
            input = torch.tensor(input).unsqueeze(0).type(torch.float64)

            
            
            if without_pred >= lookback:
                forecast_df[i+lookback: i+2*lookback] = pd.Series(model(input)[0].detach().numpy(), index=test[i+lookback: i+2*lookback].index)
                ind = forecast_df[i+lookback: i+2*lookback].index
                for k, r in seg[j].loc[(seg[j].index >= ind.min()) & (seg[j].index <= ind.max())].iterrows():
                
                    diff = np.random.normal(pt[l].at[r['Влияние'], ('mean', 'diff')], pt[l].at[r['Влияние'], ('std', 'diff')])
                    t = np.random.normal(pt[l].at[r['Влияние'], ('mean', 'max_d')], pt[l].at[r['Влияние'], ('std', 'max_d')])
                    t = min(k+timedelta(max(t, 0)), d_max)
                    forecast_df[k:t] += diff * coeff[j]
            else:
                pred = model(input)[0].detach().numpy()[:without_pred]
                forecast_df[i+lookback: i+lookback+without_pred] = pd.Series(pred, index=test[i+lookback: i+lookback+without_pred].index)
                ind = forecast_df[i+lookback: i+lookback+without_pred].index
                for k, r in seg[j].loc[(seg[j].index >= ind.min()) & (seg[j].index <= ind.max())].iterrows():
                
                    diff = np.random.normal(pt[l].at[r['Влияние'], ('mean', 'diff')], pt[l].at[r['Влияние'], ('std', 'diff')])
                    t = np.random.normal(pt[l].at[r['Влияние'], ('mean', 'max_d')], pt[l].at[r['Влияние'], ('std', 'max_d')])
                    t = min(k+timedelta(max(t, 0)), d_max)
                    forecast_df[k:t] += diff * coeff[j]
            without_pred -=step
        forecast_df = forecast_df.loc[train_border:]
        forecast_df_list.append(forecast_df.copy())

    for d in forecast_df_list[-1].index:
        forecast_df[d] = 0
        for i in range(n):
            forecast_df[d] += forecast_df_list[i][d]
        forecast_df[d] /= n
    test = test[train_border:]
    #print(forecast_df)
    res[j] = dict()
    res[j]['forecast'] = forecast_df
    res[j]['test'] = test

    mae_lstm = mean_absolute_error(res[j]['test'], res[j]['forecast'])
    mse_lstm = mean_squared_error(res[j]['test'], res[j]['forecast'])
    print(mae_lstm, mse_lstm)

In [ ]:
res_first = dict()
steps = {1: 52, 2: 52, 3: 13, 4: 2, 5: 2, 6: 26}
for j in range(1, 7):

    # обучение
    model = models[j]['model']
    test  = models[j]['test']
    train = models[j]['train']

    # прогноз
    forecast_df = pd.Series([0]*len(test), index=test.index)
    input = X_test[0]
    forecast_df[:lookback] = pd.Series(X_test[0], index=test[:lookback].index)
    #print(forecast_df[:lookback])
    
    without_pred = len(forecast_df) - lookback
    step = 1
    
    for i in range(0, len(forecast_df) - lookback, step):
        input = forecast_df[i:i+lookback]
        input = torch.tensor(input).unsqueeze(0).type(torch.float64)

        if without_pred >= lookback:
            forecast_df[i+lookback: i+2*lookback] = pd.Series(model(input)[0].detach().numpy(), index=test[i+lookback: i+2*lookback].index)
        else:
            pred = model(input)[0].detach().numpy()[:without_pred]
            forecast_df[i+lookback: i+lookback+without_pred] = pd.Series(pred, index=test[i+lookback: i+lookback+without_pred].index)
        without_pred -=step

    forecast_df = forecast_df.loc[train_border:]
    test = test[train_border:]
    res_first[j] = dict()
    res_first[j]['forecast'] = forecast_df
    res_first[j]['test'] = test

In [ ]:
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles = [data_slice[(i % 2) * 3 + i // 2 + 1]['name'] for i in range(6)],
                    vertical_spacing = 0.1, horizontal_spacing=0.05)
for i in range(6):
    fig.add_trace(go.Scatter(x=res[i + 1]['test'].index, y=res[i + 1]['test'], name='Ground truth',
                             line=dict(color='Olive'), showlegend=False), row=i%3+1, col=i//3+1)
    fig.add_trace(go.Scatter(x=res_first[i + 1]['forecast'].index, y=res_first[i + 1]['forecast'], name='Prediction',
                             line=dict(color='mediumvioletred'), showlegend=False), row=i%3+1, col=i//3+1)
    fig.add_trace(go.Scatter(x=res[i + 1]['forecast'].index, y=res[i + 1]['forecast'], name='Biased correction',
                             line=dict(color='blue'), showlegend=False), row=i%3+1, col=i//3+1)
    
fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
fig.update_annotations(font_size=12)

fig.show()

In [ ]:
print('MAE для прогноза на основе LSTL:')
for i in range(1, 7):
    mae_lstm = mean_absolute_error(res[i]['test'], res_first[i]['forecast'])
    print(f'{data_slice[i]["name"]}: {mae_lstm}')
print()
print('MAE для прогноза на основе LSTL cо смещением:')
for i in range(1, 7):
    mae_lstm = mean_absolute_error(res[i]['test'], res[i]['forecast'])
    print(f'{data_slice[i]["name"]}: {mae_lstm}')
print()
print('MSE для прогноза на основе LSTL:')
for i in range(1, 7):
    mse_lstm = mean_squared_error(res[i]['test'], res_first[i]['forecast'])
    print(f'{data_slice[i]["name"]}: {mse_lstm}')
print()
print('MSE для прогноза на основе LSTL cо смещением:')
for i in range(1, 7):
    mse_lstm = mean_squared_error(res[i]['test'], res[i]['forecast'])
    print(f'{data_slice[i]["name"]}: {mse_lstm}')
print()
print('MAPE для прогноза на основе LSTL:')
for i in range(1, 7):
    mape_lstm = mean_absolute_percentage_error(res[i]['test'], res_first[i]['forecast'])
    print(f'{data_slice[i]["name"]}: {mape_lstm}')
print()
print('MAPE для прогноза на основе LSTL cо смещением:')
for i in range(1, 7):
    mape_lstm = mean_absolute_percentage_error(res[i]['test'], res[i]['forecast'])
    print(f'{data_slice[i]["name"]}: {mape_lstm}')

# Модельный пример (рис 27-32)

In [ ]:
basin = 'Азово-Черноморский бассейн'
ports = port_df.loc[port_df['Бассейн'] == basin, 'Название порта'].to_list()
ship_type = '60 / General cargo/multi-purpose ship'
date_from=date(2022, 1, 1) 
date_to=date(2022, 12, 31)

with Session(engine) as session:
    query = session.query(
                    PortCall.port_call.label('Port'),
                    func.count(PortCall.arrival).label('Number of calls'),
                    func.sum(Ship.tonnage).label('Tonnage'),
                    func.avg((func.extract('epoch', PortCall.departure)-func.extract('epoch', PortCall.arrival))/ 3600).label('Mean deltatime, h'),
                    func.stddev((func.extract('epoch', PortCall.departure)-func.extract('epoch', PortCall.arrival))/ 3600).label('Std deltatime, h'),
                    func.max(Ship.length).label('Max length, m'),
                    func.max(Ship.width).label('Max width, m')
                    ).\
                    join(PortCall, PortCall.ship_id == Ship.id).\
                    join(Port, PortCall.port_id == Port.id).\
                    filter(Port.name.in_(tuple(ports))).\
                    filter(PortCall.arrival >= date_from).\
                    filter(PortCall.arrival <= date_to).\
                    filter(Ship.type == ship_type).\
                    filter(Ship.tonnage != np.NaN).\
                    group_by('Port').\
                    order_by('Tonnage')
    res = pd.read_sql(query.statement, query.session.bind)

res[::-1]

In [ ]:
adj_table = np.zeros((107, 107), dtype=int) # порт отправления, порт прибытия
ships = dict() # корабль, нынешний порт

date_from=date(2022, 1, 1) 
date_to=date(2022, 12, 31)

with Session(engine) as session:
        query = session.query(PortCall.ship_id, PortCall.port_id).\
                        filter(PortCall.arrival >= date_from).\
                        filter(PortCall.arrival <= date_to).\
                        order_by(PortCall.arrival.asc())
        
        for v in query:
            if v[0] in ships.keys():
                adj_table[ships[v[0]], v[1]] += 1
            ships[v[0]] = v[1]
                        
G = nx.DiGraph()
nodes = [(node, {'name': attr['Название порта'], 'basin': attr['Бассейн']}) for (node, attr) in port_df.to_dict('index').items()]
G.add_nodes_from(nodes)

edges = []
for i in range(107):
    for j in range(107):
        if adj_table[i, j] !=0:
            G.add_edge(i, j, weight=adj_table[i, j])

max_component = max(nx.weakly_connected_components(G), key=len)
    
G_max_component = nx.Graph()            
for node in max_component:  
    G_max_component.add_node(node)
for (i, j) in list(G.edges):            
    if i in max_component and j in max_component:
        G_max_component.add_edge(i, j)

G = nx.Graph(G_max_component)

degree_centrality = nx.centrality.degree_centrality(G)
for i in set(range(107)) - set(list(degree_centrality)):
    degree_centrality[i] = 0

res['Degree centrality'] = 0
for i, r in res.iterrows():
    res.at[i, 'Degree centrality'] = degree_centrality[port_df[port_df['Название порта'] == r['Port']].index[0]]

res = res.set_index('Port')
res   

In [ ]:
res_model = dict()
res['MAE'] = 0
res['MSE'] = 0

for i, r in res.iterrows():
    #условия
    print(i)
    date_from=date(2015, 1, 1) 
    date_to=date(2023, 12, 31)
    df = arrival_dynamics_with_tonnage([i], ship_type, date_from, date_to).tonnage
    df_min, df_max = df.min(), df.max()
    df = (df-df_min)/(df_max -df_min)
    df = df.resample('W').mean()
    
    # датасет разбитый на train и test
    lookback = 52
    train_border = date(2020, 12, 31)
    train, test = df.loc[:train_border], df.loc[train_border - timedelta(lookback*7):]
    X_train, y_train = create_dataset(train, lookback=lookback)
    X_test, y_test = create_dataset(test, lookback=lookback)

    # обучение
    model = model_lstm(X_train, y_train, X_test, y_test, lookback, n_epochs = 500, print_bool=True)
    res_model[i] = dict()
    res_model[i]['model'] = model

    # прогноз
    forecast_df = pd.Series([0]*len(test), index=test.index)
    input = X_test[0]
    forecast_df[:lookback] = pd.Series(X_test[0], index=test[:lookback].index)
    without_pred = len(forecast_df) - lookback

    for j in range(len(forecast_df) - lookback):
        input = forecast_df[j:j+lookback]
        input = torch.tensor(input).unsqueeze(0).type(torch.float64)
        
        if without_pred >= lookback:
            forecast_df[j+lookback: j+2*lookback] = pd.Series(model(input)[0].detach().numpy(), index=test[j+lookback: j+2*lookback].index)
        else:
            pred = model(input)[0].detach().numpy()[:without_pred]
            forecast_df[j+lookback: j+lookback+without_pred] = pd.Series(pred, index=test[j+lookback: j+lookback+without_pred].index)
        without_pred -=1

    forecast_df = forecast_df.loc[train_border:]
    test = test[train_border:]
    mae = mean_absolute_error(test, forecast_df)
    mse = mean_squared_error(test, forecast_df)
    res_model[i]['MAE'] = mae
    res_model[i]['MSE'] = mse
    res.at[i, 'MAE'] = mae
    res.at[i,'MSE'] = mse
    print(f'MAE: {mae}, MSE: {mse}')

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df, name='prediction'))
    fig.add_trace(go.Scatter(x=test.index, y=test, name='test'))
    fig.show()

In [ ]:
res

In [ ]:
res_norm = res.copy()
imp_col = {'Number of calls': 1, 'Tonnage': 1, 'Mean deltatime, h': -1, 'Std deltatime, h': -1, 'Max length, m': 1, 'Max width, m': 1, 'Degree centrality': 1, 'MAE': -1, 'MSE': -1}
for col in res.columns:
    if imp_col[col] == 1:
        res_norm[col] = (res_norm[col] - res_norm[col].min())/(res_norm[col].max() - res_norm[col].min())
    else:
         res_norm[col] = (res_norm[col].max() - res_norm[col])/(res_norm[col].max() - res_norm[col].min())
res_norm

In [ ]:
# row =  [1/9] * 9 
row = [3/25, 2/25, 2/25, 5/25, 0, 0, 1/25, 6/25, 6/25]
weight_of_indicators = pd.DataFrame([row], columns=res.columns)
weight_of_indicators

In [ ]:
res_norm['Сonvolution'] = 0
for i, col in enumerate(res.columns):
    res_norm['Сonvolution'] += (res_norm[col] * row[i])
res_norm.sort_values('Сonvolution', ascending=False)